# Evaluation: Syntax

Part I of the computational evaluation of AI-generated linked data for [Linking Anthropology's Data and Archives (LADA)](https://ischool.umd.edu/projects/building-a-sustainable-future-for-anthropologys-archives-researching-primary-source-data-lifecycles-infrastructures-and-reuse/), focused on syntax (e.g., do the metadata adhere to the expected serialization formats?).

---

**Table of Contents:**

I. [Data Loading](#data-loading)

II. [Syntax](#syntax)

  * [XML](#xml)
  
  * [JSON](#json)

---

## Data Loading

In [2]:
import utils
import config
import pandas as pd
import xml.etree.ElementTree as ET
import json
from lxml import etree
from pathlib import Path
import os
import re

# sax - to validate XML well-formed
# xml.etree.ElementTree - to validate text between tags
# xml.etree.ElementTree + xml.etree.ElementTree.XMLSchema's validate() - to validate XML well-formed
# lxml etree.XMLParser - to validate well-formed based on input XML schema
# json_checker - to validate Python data types (incl. but not limited to those obtained from JSON)
# jsonschema.validate
# ShEx - for RDF graphs, ShExJ for JSON - NOTE: couldn't install package
# OntoME - for CIDOC-CRM ontology alignment

Create variables to reference existing directories and files.

In [3]:
dublin_path = "cleaned/dublin_core/"  # XML data files
schema_path = "cleaned/schema_org/"   # JSON data files
cidoc_path = "cleaned/cidoc_crm/"     # JSON data files

dublin_t1_dir = config.task1_data+dublin_path
schema_t1_dir = config.task1_data+schema_path
cidoc_t1_dir = config.task1_data+cidoc_path

dublin_p1_dir = config.playgrd1_data+dublin_path
schema_p1_dir = config.playgrd1_data+schema_path
cidoc_p1_dir = config.playgrd1_data+cidoc_path

dublin_p3_dir = config.playgrd3_data+dublin_path
schema_p3_dir = config.playgrd3_data+schema_path
cidoc_p3_dir = config.playgrd3_data+cidoc_path

Create a folder to store the error reports in.

In [ ]:
d = "syntax"
report_dir = f"data/error_reports/{d}/"
Path(report_dir).mkdir(parents=True, exist_ok=True)

## Syntax

### XML

First, read and evaluate only the files with a `.xml` extension.

In [5]:
extension = ".xml"
dublin_file_paths = []
dublin_files_t1 = [f for f in os.listdir(dublin_t1_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_t1_dir+f for f in dublin_files_t1]
dublin_files_p1 = [f for f in os.listdir(dublin_p1_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_p1_dir+f for f in dublin_files_p1]
dublin_files_p3 = [f for f in os.listdir(dublin_p3_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_p3_dir+f for f in dublin_files_p3]
dublin_file_paths.sort()
total_dcxml_files = len(dublin_file_paths)
print(f"Total Dublin Core {extension[1:].upper()} files:", total_dcxml_files)

Total Dublin Core XML files: 107


In [6]:
syntax_errors, errored_files = [], []
for file_path in dublin_file_paths:
    try:
        tree = etree.parse(file_path)
    except Exception as e:
        f_error = {"file": file_path, "exception_type": type(e), "exception_message": str(e)}
        syntax_errors += [f_error]
        errored_files += [file_path]
print("Files with errors:", 
      len(errored_files), "of", total_dcxml_files,
      f"({(len(errored_files)/total_dcxml_files)*100:.2f}%)")

Files with errors: 43 of 107 (40.19%)


In [7]:
df_se = pd.DataFrame.from_dict(syntax_errors)
pattern = "^[\D]+,"
new_exception_col = df_se["exception_message"].apply(lambda x: re.findall(pattern, x)[0][:-1])
df_se.insert(len(df_se.columns)-1, "exception_subtype", new_exception_col)
df_se.head()

<>:2: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\D'
/var/folders/9c/dtt4chrx6lgcgnfz6r_z5ymm0000gn/T/ipykernel_2293/179748656.py:2: SyntaxWarning: invalid escape sequence '\D'
  pattern = "^[\D]+,"


,file,exception_type,exception_subtype,exception_message
0,data/data_playground_task1/cleaned/dublin_core...,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix dc on title is not defined,"Namespace prefix dc on title is not defined, l..."
1,data/data_playground_task1/cleaned/dublin_core...,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix dc on title is not defined,"Namespace prefix dc on title is not defined, l..."
2,data/data_playground_task1/cleaned/dublin_core...,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix dc on title is not defined,"Namespace prefix dc on title is not defined, l..."
3,data/data_playground_task1/cleaned/dublin_core...,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix dc on title is not defined,"Namespace prefix dc on title is not defined, l..."
4,data/data_playground_task1/cleaned/dublin_core...,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix dc on title is not defined,"Namespace prefix dc on title is not defined, l..."


Next, evaluate every record by reading the TXT files to:
- check whether relevnt DC namespace(s) are present
- check whether RDF namespace is present
- check whether a prolog is present
- check whether a prolog with UTF-8 encoding is present

In [8]:
extension = ".txt"
dublin_file_paths = []
dublin_files_t1 = [f for f in os.listdir(dublin_t1_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_t1_dir+f for f in dublin_files_t1]
dublin_files_p1 = [f for f in os.listdir(dublin_p1_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_p1_dir+f for f in dublin_files_p1]
dublin_files_p3 = [f for f in os.listdir(dublin_p3_dir) if f.endswith(extension)]
dublin_file_paths += [dublin_p3_dir+f for f in dublin_files_p3]
dublin_file_paths.sort()
total_dctxt_files = len(dublin_file_paths)
print(f"Total Dublin Core {extension[1:].upper()} files:", total_dctxt_files)

Total Dublin Core TXT files: 107


In [9]:
custom_syntax_errors, more_errored_files = [], []
for file_path in dublin_file_paths:
    with open(file_path, "r") as f:
        f_string = f.read()
        file_path = file_path.replace(".txt", ".xml")
        if not utils.hasDCNamespaces(f_string):
            f_error = {"file": file_path, "exception_type": "Custom syntax check", "exception_subtype": "Missing namespace", "exception_message": "Missing Dublin Core namespace(s)"}
            custom_syntax_errors += [f_error]
            more_errored_files += [file_path]
        if not utils.hasRDFNamespace(f_string):
            f_error = {"file": file_path, "exception_type": "Custom syntax check", "exception_subtype": "Missing namespace", "exception_message": "Missing RDF namespace"}
            custom_syntax_errors += [f_error]
            more_errored_files += [file_path]
        if not utils.hasProlog(f_string):
            f_error = {"file": file_path, "exception_type": "Custom syntax check", "exception_subtype": "Missing prolog", "exception_message": "Missing prolog"}
            custom_syntax_errors += [f_error]
            more_errored_files += [file_path]
        if not utils.hasPrologWithEncoding(f_string):
            f_error = {"file": file_path, "exception_type": "Custom syntax check", "exception_subtype": "Missing prolog", "exception_message": "Missing prolog with UTF-8 encoding"}
            custom_syntax_errors += [f_error]
            more_errored_files += [file_path]
print(len(custom_syntax_errors), "additional syntax errors found across", len(set(more_errored_files)), "out of", total_dctxt_files, "files")

334 additional syntax errors found across 107 out of 107 files


In [10]:
df_se = pd.concat([df_se, pd.DataFrame.from_dict(custom_syntax_errors)])
new_file_col = df_se["file"].apply(lambda x: x.split("/")[-1])
df_se = df_se.rename(columns={"file":"file_path"})
df_se.insert(1, "file_name", new_file_col)
df_se.tail()

,file_path,file_name,exception_type,exception_subtype,exception_message
329,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_027.xml,Custom syntax check,Missing prolog,Missing prolog with UTF-8 encoding
330,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_028.xml,Custom syntax check,Missing namespace,Missing Dublin Core namespace(s)
331,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_028.xml,Custom syntax check,Missing namespace,Missing RDF namespace
332,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_028.xml,Custom syntax check,Missing prolog,Missing prolog
333,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_028.xml,Custom syntax check,Missing prolog,Missing prolog with UTF-8 encoding


In [11]:
df_se.exception_type.value_counts()

exception_type
Custom syntax check                    334
<class 'lxml.etree.XMLSyntaxError'>     43
Name: count, dtype: int64

In [12]:
custom_message_report = pd.DataFrame(df_se.loc[df_se.exception_type == "Custom syntax check"].exception_message.value_counts())
custom_df = df_se.loc[df_se.exception_type == "Custom syntax check"]
custom_df = custom_df[["exception_type", "exception_subtype", "exception_message"]].drop_duplicates()
custom_df = custom_df.set_index("exception_message").join(custom_message_report).reset_index()
custom_report = custom_df[["exception_type", "exception_subtype", "exception_message", "count"]]
custom_report

,exception_type,exception_subtype,exception_message,count
0,Custom syntax check,Missing namespace,Missing Dublin Core namespace(s),56
1,Custom syntax check,Missing namespace,Missing RDF namespace,107
2,Custom syntax check,Missing prolog,Missing prolog,64
3,Custom syntax check,Missing prolog,Missing prolog with UTF-8 encoding,107


The `count` column refers to the total occurrence of each exception, so the sum of that column may exceed the total number of files.

In [13]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "syntax_error_stats_custom_subtypes"
custom_report.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=False
    )

In [14]:
xml_syntax_subtypes = pd.DataFrame(df_se.loc[df_se.exception_type != "Custom syntax check"].exception_subtype.value_counts())
syntax_df = df_se.loc[df_se.exception_type != "Custom syntax check"]
syntax_df = syntax_df[["exception_type", "exception_subtype"]].drop_duplicates()
syntax_df = syntax_df.set_index("exception_subtype").join(xml_syntax_subtypes).reset_index()
xml_syntax_error_report = syntax_df[["exception_type", "exception_subtype", "count"]]
xml_syntax_error_report

,exception_type,exception_subtype,count
0,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix dc on title is not defined,33
1,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix rdf for about on Description ...,7
2,<class 'lxml.etree.XMLSyntaxError'>,xmlns:dc: Empty XML namespace is not allowed,1
3,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix rdf on Description is not def...,1
4,<class 'lxml.etree.XMLSyntaxError'>,xmlParseEntityRef: no name,1


In [15]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "syntax_error_stats__subtypes"
xml_syntax_error_report.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=False
    )

In [16]:
df_se_subtypes = df_se[["file_path", "file_name", "exception_type", "exception_subtype"]].drop_duplicates()
df_se_subtypes.tail()

,file_path,file_name,exception_type,exception_subtype
324,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_026.xml,Custom syntax check,Missing prolog
326,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_027.xml,Custom syntax check,Missing namespace
328,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_027.xml,Custom syntax check,Missing prolog
330,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_028.xml,Custom syntax check,Missing namespace
332,data/data_task1/cleaned/dublin_core/dc_record_...,dc_record_028.xml,Custom syntax check,Missing prolog


In [17]:
subtype_report = pd.DataFrame(df_se_subtypes[["exception_type", "exception_subtype"]].value_counts())
subtype_report = subtype_report.rename(columns={"count":"file_count"})
subtype_report


file_count
exception_type                      exception_subtype                                             
Custom syntax check                 Missing namespace                                          107
                                    Missing prolog                                             107
<class 'lxml.etree.XMLSyntaxError'> Namespace prefix dc on title is not defined                 33
                                    Namespace prefix rdf for about on Description i...           7
                                    Namespace prefix rdf on Description is not defined           1
                                    xmlParseEntityRef: no name                                   1
                                    xmlns:dc: Empty XML namespace is not allowed                 1

In [18]:
all_errored_files = set(errored_files + more_errored_files)
totals_report = pd.DataFrame({
    "exception_type": ["TOTAL FILES", "FILES WITH EXCEPTION"],
    "exception_subtype": ["NA", "NA"],
    "file_count": [total_dcxml_files, len(all_errored_files)]
    })
totals_report

,exception_type,exception_subtype,file_count
0,TOTAL FILES,NA,107
1,FILES WITH EXCEPTION,NA,107


In [19]:
more_errored_files[0]

'data/data_playground_task1/cleaned/dublin_core/dc_record_003.xml'

In [24]:
xml_report = pd.concat([subtype_report.reset_index(), totals_report])
proportions = (xml_report[["file_count"]]/total_dcxml_files).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
xml_report.insert(len(xml_report.columns), "proportion_of_all_files", percentages)
xml_report = xml_report.reset_index().drop(columns=["index"])
xml_report

,exception_type,exception_subtype,file_count,proportion_of_all_files
0,Custom syntax check,Missing namespace,107,100.00%
1,Custom syntax check,Missing prolog,107,100.00%
2,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix dc on title is not defined,33,30.84%
3,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix rdf for about on Description ...,7,6.54%
4,<class 'lxml.etree.XMLSyntaxError'>,Namespace prefix rdf on Description is not def...,1,0.93%
5,<class 'lxml.etree.XMLSyntaxError'>,xmlParseEntityRef: no name,1,0.93%
6,<class 'lxml.etree.XMLSyntaxError'>,xmlns:dc: Empty XML namespace is not allowed,1,0.93%
7,TOTAL FILES,NA,107,100.00%
8,FILES WITH EXCEPTION,NA,107,100.00%


Save the reports as CSV files.

In [25]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "syntax_error_stats"
xml_report.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=False
    )

In [26]:
metadata_standard = "dublin_core"
data_serialization = "xml"
report_type = "syntax_errors"
df_se.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=False
    )

### JSON

First, read and evaluate only the files with a `.json` extension.

In [5]:
extension = ".json"
cidoc_file_paths = []
cidoc_files_t1 = [f for f in os.listdir(cidoc_t1_dir) if f.endswith(extension)]
cidoc_file_paths += [cidoc_t1_dir+f for f in cidoc_files_t1]
cidoc_files_p1 = [f for f in os.listdir(cidoc_p1_dir) if f.endswith(extension)]
cidoc_file_paths += [cidoc_p1_dir+f for f in cidoc_files_p1]
cidoc_files_p3 = [f for f in os.listdir(cidoc_p3_dir) if f.endswith(extension)]
cidoc_file_paths += [cidoc_p3_dir+f for f in cidoc_files_p3]
cidoc_file_paths.sort()
print("Total CIDOC-CRM JSON files:", len(cidoc_file_paths))

Total CIDOC-CRM JSON files: 97


In [6]:
cidoc_file_paths[0]

'data/data_playground_task1/cleaned/cidoc_crm/cidoccrm_record_003.json'

In [7]:
extension = ".json"
schema_file_paths = []
schema_files_t1 = os.listdir(schema_t1_dir)
schema_file_paths += [schema_t1_dir+f for f in schema_files_t1 if f.endswith(extension)]
schema_files_p1 = os.listdir(schema_p1_dir)
schema_file_paths += [schema_p1_dir+f for f in schema_files_p1 if f.endswith(extension)]
schema_files_p3 = os.listdir(schema_p3_dir)
schema_file_paths += [schema_p3_dir+f for f in schema_files_p3 if f.endswith(extension)]
schema_file_paths.sort()
print("Total Schema.org JSON files:", len(schema_file_paths))

Total Schema.org JSON files: 116


In [8]:
schema_file_paths[0]

'data/data_playground_task1/cleaned/schema_org/sdo_record_003.json'

In [9]:
json_file_paths = cidoc_file_paths + schema_file_paths
total_json_files = len(json_file_paths)
print(len(json_file_paths))

213


In [10]:
syntax_errors = []
for json_f in json_file_paths:
    with open(json_f) as f:
        try:
            data = json.load(f)
        except Exception as e:
            f_error = {"file": json_f, "exception_type": type(e), "exception_message": str(e)}
            syntax_errors += [f_error]
        f.close()
print(
    "Files with errors:", 
    len(syntax_errors), "of", len(json_file_paths),
    f"({(len(syntax_errors)/len(json_file_paths))*100:.2f}%)"
    )

Files with errors: 4 of 213 (1.88%)


In [11]:
df_se = pd.DataFrame.from_dict(syntax_errors)
new_file_col = df_se["file"].apply(lambda x: x.split("/")[-1])
df_se = df_se.rename(columns={"file":"file_path"})
df_se.insert(1, "file_name", new_file_col)
df_se

,file_path,file_name,exception_type,exception_message
0,data/data_playground_task1/cleaned/cidoc_crm/c...,cidoccrm_record_018.json,<class 'json.decoder.JSONDecodeError'>,"Expecting ',' delimiter: line 99 column 5 (cha..."
1,data/data_playground_task1/cleaned/cidoc_crm/c...,cidoccrm_record_070.json,<class 'json.decoder.JSONDecodeError'>,"Expecting ',' delimiter: line 35 column 1 (cha..."
2,data/data_playground_task1/cleaned/schema_org/...,sdo_record_018.json,<class 'json.decoder.JSONDecodeError'>,"Expecting ',' delimiter: line 27 column 1 (cha..."
3,data/data_task1/cleaned/schema_org/sdo_record_...,sdo_record_006.json,<class 'json.decoder.JSONDecodeError'>,Expecting ':' delimiter: line 2 column 5 (char 6)


In [12]:
df_se.exception_type.unique()

array([<class 'json.decoder.JSONDecodeError'>], dtype=object)

In [13]:
df_se.exception_message.unique()

array(["Expecting ',' delimiter: line 99 column 5 (char 3125)",
       "Expecting ',' delimiter: line 35 column 1 (char 1407)",
       "Expecting ',' delimiter: line 27 column 1 (char 914)",
       "Expecting ':' delimiter: line 2 column 5 (char 6)"], dtype=object)

Looking at the files that triggered the above error messages, it seems that often what's missing is the final curly brace.  Also, in one file, the quotes surrounding strings were doubled (e.g., `""@context"":...` instead of `"@context"`) and in another file, an attempt at a comment was made using `//`.  While the error messages are useful in locating the general source of the error within a file, they're less informative for distinguishing what needs to be changed to correct the file.

Export a reports about the JSON errors.

In [14]:
subdf_se = pd.DataFrame(df_se[["file_path", "file_name", "exception_type"]].drop_duplicates())
subdf_se.tail()

,file_path,file_name,exception_type
0,data/data_playground_task1/cleaned/cidoc_crm/c...,cidoccrm_record_018.json,<class 'json.decoder.JSONDecodeError'>
1,data/data_playground_task1/cleaned/cidoc_crm/c...,cidoccrm_record_070.json,<class 'json.decoder.JSONDecodeError'>
2,data/data_playground_task1/cleaned/schema_org/...,sdo_record_018.json,<class 'json.decoder.JSONDecodeError'>
3,data/data_task1/cleaned/schema_org/sdo_record_...,sdo_record_006.json,<class 'json.decoder.JSONDecodeError'>


In [15]:
type_report = pd.DataFrame(df_se[["exception_type"]].value_counts())
type_report = type_report.rename(columns={"count":"file_count"})
type_report

,file_count
exception_type,
<class 'json.decoder.JSONDecodeError'>,4


In [18]:
totals_report = pd.DataFrame({
    "exception_type": ["TOTAL FILES", "FILES WITH EXCEPTION"],
    "file_count": [len(json_file_paths), len(syntax_errors)]
    })

In [19]:
json_report = pd.concat([type_report.reset_index(), totals_report])
proportions = (json_report[["file_count"]]/(len(json_file_paths))).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
json_report.insert(len(json_report.columns), "proportion_of_all_files", percentages)
json_report = json_report.reset_index().drop(columns=["index"])
json_report


,exception_type,file_count,proportion_of_all_files
0,<class 'json.decoder.JSONDecodeError'>,4,1.88%
1,TOTAL FILES,213,100.00%
2,FILES WITH EXCEPTION,4,1.88%


Calculate how many files are meant to adhere to Schema.org and CIDOC-CRM models.

In [20]:
sdo_error_df = subdf_se.loc[subdf_se.file_name.str.contains("sdo")]
cidoc_error_df = subdf_se.loc[subdf_se.file_name.str.contains("cidoccrm")]

In [21]:
exception_types = df_se.exception_type.unique()
assert len(exception_types) == 1, "There are multiple exception types in the JSON files: {}".format(exception_types)
df_type = df_se.loc[df_se.exception_type == exception_types[0]]
sdo_type_df = df_type.loc[df_type.file_name.str.contains("sdo")]
cidoc_type_df = df_type.loc[df_type.file_name.str.contains("cidoccrm")]

In [22]:
json_report.insert(len(json_report.columns), "schema_org_files", [sdo_type_df.shape[0], len(schema_file_paths), sdo_error_df.shape[0]])
proportions = (json_report[["schema_org_files"]]/(len(schema_file_paths))).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
json_report.insert(len(json_report.columns), "proportion_of_schema_files", percentages)
proportions = (json_report[["schema_org_files"]]/(len(json_file_paths))).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
json_report.insert(len(json_report.columns), "schema_proportion_of_all_files", percentages)

json_report.insert(len(json_report.columns), "cidoc_crm_files", [cidoc_type_df.shape[0], len(cidoc_file_paths), cidoc_error_df.shape[0]])
proportions = (json_report[["cidoc_crm_files"]]/(len(cidoc_file_paths))).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
json_report.insert(len(json_report.columns), "proportion_of_cidoc_files", percentages)
proportions = (json_report[["cidoc_crm_files"]]/(len(json_file_paths))).values
percentages = [f"{proportion[0]*100:.2f}%" for proportion in proportions]
json_report.insert(len(json_report.columns), "cidoc_proportion_of_all_files", percentages)

json_report

,exception_type,file_count,proportion_of_all_files,schema_org_files,proportion_of_schema_files,schema_proportion_of_all_files,cidoc_crm_files,proportion_of_cidoc_files,cidoc_proportion_of_all_files
0,<class 'json.decoder.JSONDecodeError'>,4,1.88%,2,1.72%,0.94%,2,2.06%,0.94%
1,TOTAL FILES,213,100.00%,116,100.00%,54.46%,97,100.00%,45.54%
2,FILES WITH EXCEPTION,4,1.88%,2,1.72%,0.94%,2,2.06%,0.94%


Save the reports as CSV files.

In [23]:
metadata_standard = "cidoc-and-sdo"
data_serialization = "json"

In [24]:
report_type = "syntax_error_stats"
json_report.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=False
    )

In [25]:
report_type = "syntax_errors"
df_se.to_csv(
    report_dir+"{metadata_standard}_{data_serialization}_{report_type}.csv".format(
        metadata_standard=metadata_standard,
        data_serialization=data_serialization,
        report_type=report_type
        ), index=False
    )